# Agregación y having
### **Ingeniería de datos**
**Profesor: Domagoj Vrgoč**

### Introducción

En esta clase continuaremos con nuestro análisis de la base de datos de actores y peliculas. Para esto ocuparemos agregación, agrupamiento, y distintos filtros que uno puede imponer a los grupos ocupando el comando `HAVING`.

### Requisitos

Para esta actividad, así como en las siguientes actividades de SQL vamos a utilizar *Google colab* (https://colab.research.google.com), que es un entorno virtual permitiendo armar un servidor de bases de datos, y conectarse con este servidor. Para la conexión ocuparemos la herramienta llamada Jupyter Notebooks. Esta herramienta permite conectarse con un servidor SQL de la misma manera cómo hacerlo a través de la consola en un servidor local. 

El motor de bases de datos que ocuparemos en este curso se llama PostgreSQL, y uno siempre puede instalarlo localmente en su computador. Idea de ocupar Google colab es saltarse este paso, y no tener problemas con instalar, habilitar, o correr un motor de bases de datos.

Por lo tanto, para una actividad de SQL, en este curso siempre ocuparemos Jupyter Notebooks con Google colab. Para esto, se les entregará un archivo con extensión .ipynb, cual hay que subir a la plataforma Google Colab. Al inicio del tutorial mostraremos cómo funciona este proceso.


### Esquema

Para esta actividad vamos a trabajar con el siguiente esquema:

- `Peliculas(pid, pnombre, paño, pcategoria, pcalificacion, pdirector)`

- `Actores(aid, anombre, aedad)`

- `actuo_en(aid, pid, rol)`

Que corresponde a películas, actores, y la información de los roles interpretados por una actor en una película.

Las llaves en nuestro caso son:
1. `pid`, para `Peliculas`
2. `aid` para `Actores`
3. `(aid,pid,rol)` para `actuo_en`.

Es importante notar que en este caso `rol` forma parte de la llave primaria de la relación `actuo_en`, dado que debemos modelar el caso donde un actor juega dos roles distintos en una misma película.

## Tutorial

Lo primero que hay que hacer es subir este notebook a https://colab.research.google.com

### Iniciar el servidor

Para iniciar el servidor virtual, *instalar* la base de datos postgres debe correr el siguiente bloque:

In [1]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [748 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: @postgres'

### Creando la base de datos

Para crear y poblar nuestra base de datos, corramo el siguiente bloque de código:

In [2]:
%%sql

DROP TABLE IF EXISTS Peliculas;
DROP TABLE IF EXISTS Actores;
DROP TABLE IF EXISTS Actuo_En;

CREATE TABLE IF NOT EXISTS Peliculas(
    pid int PRIMARY KEY,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30),
    pcalificacion float,
    pdirector varchar(30)
);

CREATE TABLE Actores(
    aid int PRIMARY KEY,
    anombre varchar(30),
    aedad int
);

CREATE TABLE actuo_en(
    aid int,
    pid int,
    rol varchar(30),
    PRIMARY KEY (aid,pid,rol)
);

INSERT INTO Peliculas VALUES(1,'Avengers:Endgame',2019,'SciFi',8.4,'Brothers Russo');
INSERT INTO Peliculas VALUES(2,'Captain America: Civil War',2016,'SciFi',8.7,'Brothers Russo');
INSERT INTO Peliculas VALUES(3,'Iron Man',2008,'SciFi',9.1,'John Favreu');
INSERT INTO Peliculas VALUES(4,'Batman: The Dark Knight',2008,'Thriller',8.3,'Christoper Nolan');
INSERT INTO Peliculas VALUES(5,'Batman: The Dark Knight Rises',2012,'Thriller',7.9,'Christoper Nolan');
INSERT INTO Peliculas VALUES(6,'Interstellar',2014,'Drama',8.4,'Christoper Nolan');
INSERT INTO Peliculas VALUES(7,'Sherlock Holmes',2009,'Mystery',7.8,'Guy Ritchie');
INSERT INTO Peliculas VALUES(8,'Avengers: Age of Ultron',2015,'SciFi',8.3,'Joss Whedon');
INSERT INTO Peliculas VALUES(9,'Doctor Strange',2016,'SciFi',8.8,'Scott Derrickson');

INSERT INTO Actores VALUES(1,'Robert Downey Jr.',57);
INSERT INTO Actores VALUES(2,'Scarlett Johansson',37);
INSERT INTO Actores VALUES(3,'Chris Evans',40);
INSERT INTO Actores VALUES(4,'Christian Bale',48);
INSERT INTO Actores VALUES(5,'Anne Hathaway',39);
INSERT INTO Actores VALUES(6,'Paul Bettany',50);
INSERT INTO Actores VALUES(7,'Benedict Cumberbatch',45);

INSERT INTO actuo_en VALUES(1,1,'Tony Stark');
INSERT INTO actuo_en VALUES(1,2,'Tony Stark');
INSERT INTO actuo_en VALUES(1,3,'Tony Stark');
INSERT INTO actuo_en VALUES(1,7,'Sherlock Holmes');
INSERT INTO actuo_en VALUES(2,1,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(2,2,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(3,1,'Steve Rogers');
INSERT INTO actuo_en VALUES(3,2,'Steve Rogers');
INSERT INTO actuo_en VALUES(4,4,'Bruce Wayne');
INSERT INTO actuo_en VALUES(4,5,'Bruce Wayne');
INSERT INTO actuo_en VALUES(5,5,'Selina Kyle');
INSERT INTO actuo_en VALUES(5,6,'Amelia Brand');
INSERT INTO actuo_en VALUES(6,8,'J.A.R.V.I.S.');
INSERT INTO actuo_en VALUES(6,8,'Vision');
INSERT INTO actuo_en VALUES(7,9,'Doctor Strange');
INSERT INTO actuo_en VALUES(7,9,'Dormammu');
INSERT INTO actuo_en VALUES(1,8,'Tony Stark');
INSERT INTO actuo_en VALUES(3,8,'Steve Rogers');
INSERT INTO actuo_en VALUES(2,8,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(6,3,'J.A.R.V.I.S.');


 * postgresql+psycopg2://@/postgres


""


Ahora podemos visualizar el contenido de las tablas:

In [3]:
%%sql

SELECT * FROM Peliculas;

 * postgresql+psycopg2://@/postgres


,pid,pnombre,paño,pcategoria,pcalificacion,pdirector
0,1,Avengers:Endgame,2019,SciFi,8.4,Brothers Russo
1,2,Captain America: Civil War,2016,SciFi,8.7,Brothers Russo
2,3,Iron Man,2008,SciFi,9.1,John Favreu
3,4,Batman: The Dark Knight,2008,Thriller,8.3,Christoper Nolan
4,5,Batman: The Dark Knight Rises,2012,Thriller,7.9,Christoper Nolan
5,6,Interstellar,2014,Drama,8.4,Christoper Nolan
6,7,Sherlock Holmes,2009,Mystery,7.8,Guy Ritchie
7,8,Avengers: Age of Ultron,2015,SciFi,8.3,Joss Whedon
8,9,Doctor Strange,2016,SciFi,8.8,Scott Derrickson


In [4]:
%%sql

SELECT * FROM Actores;

 * postgresql+psycopg2://@/postgres


,aid,anombre,aedad
0,1,Robert Downey Jr.,57
1,2,Scarlett Johansson,37
2,3,Chris Evans,40
3,4,Christian Bale,48
4,5,Anne Hathaway,39
5,6,Paul Bettany,50
6,7,Benedict Cumberbatch,45


In [5]:
%%sql

SELECT * FROM actuo_en;

 * postgresql+psycopg2://@/postgres


,aid,pid,rol
0,1,1,Tony Stark
1,1,2,Tony Stark
2,1,3,Tony Stark
3,1,7,Sherlock Holmes
4,2,1,Natasha Romanoff
5,2,2,Natasha Romanoff
6,3,1,Steve Rogers
7,3,2,Steve Rogers
8,4,4,Bruce Wayne
9,4,5,Bruce Wayne


### Analizando la base de datos con consultas de agregación

Ahora que conocemos a nuestros datos, practicaremos el uso de agregación para analizarlos en más detalles. Adicionalmente, introduciremos la clúsula `HAVING` que nos permite controlar las propiedades de los grupos definidos por `GROUP BY`.

### Pregunta 1

Primero nos enfocaremos en la tabla de películas. Primero que queremos saber es, para cada director, el número de películas que dirigió.

Una manera de construir cosultas de agregación, es pensar en que grupos queremos definir. En nuestro caso, queremos agregar peliculas agrupadas por director. Despues, en cada grupo, simplemente queremos contar el número de filas.

In [9]:
%%sql

SELECT Peliculas.pdirector, COUNT(*)
FROM Peliculas
GROUP BY Peliculas.pdirector

 * postgresql+psycopg2://@/postgres


,pdirector,count
0,Brothers Russo,2
1,Joss Whedon,1
2,Guy Ritchie,1
3,John Favreu,1
4,Scott Derrickson,1
5,Christoper Nolan,3


Muchas veces queremos restringir los grupos por considerar. Por ejemplo, podemos exijir que cada grupo tiene al menos dos películas, o que el promedio de las películas en el grupo sea mayor que 8.5.

Para especificar la propiedad que un grupo definido en `GROUP BY` debe tener, aplicamos la clúsula `HAVING`, que opera al nivel de cada grupo. Por ejemplo, si queremos solo el número de películas para los directores con al menos dos películas, podemos escribir:

In [10]:
%%sql

SELECT Peliculas.pdirector, COUNT(*)
FROM Peliculas
GROUP BY Peliculas.pdirector
HAVING COUNT(*) >= 2

 * postgresql+psycopg2://@/postgres


,pdirector,count
0,Brothers Russo,2
1,Christoper Nolan,3


Lo que ocurre aquí es que a cada grupo se aplica un `COUNT(*)`, y si este operador devuelve un valor menor qué 2, el grupo no será considerado en el resultado final.

Para devolver el promedio de calificaiones de peliculas de cada director, la consulta es muy parecida:

In [55]:
%%sql

SELECT Peliculas.pdirector, AVG(Peliculas.pcalificacion)
FROM Peliculas
GROUP BY Peliculas.pdirector

 * postgresql+psycopg2://@/postgres


,pdirector,avg
0,Brothers Russo,8.55
1,Joss Whedon,8.30
2,Guy Ritchie,7.80
3,John Favreu,9.10
4,Scott Derrickson,8.80
5,Christoper Nolan,8.20


### Pregunta 2

Ahora exploraremos las películas de `Christopher Nolan`. En particular, nos interesa conseguir la mayor calificación de todas sus películas. Para esto, podemos ocupar:

In [12]:
%%sql

SELECT MAX(Peliculas.pcalificacion)
FROM Peliculas
WHERE Peliculas.pdirector = 'Christoper Nolan'

 * postgresql+psycopg2://@/postgres


,max
0,8.4


Uno puede agregar más codiciones a su búsqueda, o mezclar agregación con joins. Por ejemplo, si nos interesa la mayor calificcación de una película dirigida por `Christopher Nolan`, y dónde actua `Christian Bale`, podemos escribir:

In [14]:
%%sql

SELECT MAX(Peliculas.pcalificacion)
FROM Peliculas, Actores, actuo_en
WHERE Peliculas.pdirector = 'Christoper Nolan' AND 
      Peliculas.pid = actuo_en.pid AND
      actuo_en.aid = Actores.aid AND
      Actores.anombre = 'Christian Bale'

 * postgresql+psycopg2://@/postgres


,max
0,8.3


*Observación:* para sacar el nómbre de dicha película podemos ocupar el truco explicado en el tutotial previo. La mejor manera de hacer esto explicaremos en la siguiente clase.

### Pregunta 3

Para finalizar nuestro análisis de las películas, nos interesa conseguir el número de actores actuando en cada película. Un primer intento de hacer esto puede ser lo siguiente:

In [21]:
%%sql

SELECT actuo_en.pid, COUNT(*)
FROM actuo_en
GROUP BY actuo_en.pid

 * postgresql+psycopg2://@/postgres


,pid,count
0,9,2
1,3,2
2,5,2
3,4,1
4,6,1
5,2,3
6,7,1
7,1,3
8,8,5


Un problema con esta solución es que, por ejemplo, la película con el `id = 9` es reportada con dos actores, pese que solo tenemos la información que `Benedict Cumberbatch` actua en ella. El problema es que este actor interpreta dos roles en la misma película. Para devolver el número correcto, debemos ocupar `DISTINCT` al nivel de atributo `aid`:

In [22]:
%%sql

SELECT actuo_en.pid, COUNT(DISTINCT actuo_en.aid)
FROM actuo_en
GROUP BY actuo_en.pid

 * postgresql+psycopg2://@/postgres


,pid,count
0,1,3
1,2,3
2,3,2
3,4,1
4,5,2
5,6,1
6,7,1
7,8,4
8,9,1


Cómo podemos observar, el resultado es ahora correcto.

Si adicionalmente queremos devolver el nombre de cada película, debemos hacer el join con la tabla `Películas`, y agregar este atributo a `GROUP BY`:

In [26]:
%%sql

SELECT Peliculas.pnombre, actuo_en.pid, COUNT(DISTINCT actuo_en.aid)
FROM actuo_en, Peliculas
WHERE actuo_en.pid = Peliculas.pid
GROUP BY actuo_en.pid, Peliculas.pnombre

 * postgresql+psycopg2://@/postgres


,pnombre,pid,count
0,Avengers:Endgame,1,3
1,Captain America: Civil War,2,3
2,Iron Man,3,2
3,Batman: The Dark Knight,4,1
4,Batman: The Dark Knight Rises,5,2
5,Interstellar,6,1
6,Sherlock Holmes,7,1
7,Avengers: Age of Ultron,8,4
8,Doctor Strange,9,1


Al nivel intuitivo, probablemente nos hubiese gustado escribir algo cómo:

```SQL
SELECT Peliculas.pnombre, COUNT(*)
FROM actuo_en, Peliculas
WHERE actuo_en.pid = Peliculas.pid
GROUP BY actuo_en.pid
```

El problema con esto es que `Peliculas.pnombre` no está en la definición del grupo. La regla general es que cada atributo que sale en la proyección debe estar en la definición del grupo. Si no, Postgres nos arrojará un error.

Simplificando un poco nuestra consulta, podemos escribir:

In [27]:
%%sql

SELECT Peliculas.pnombre, COUNT(DISTINCT actuo_en.aid)
FROM actuo_en, Peliculas
WHERE actuo_en.pid = Peliculas.pid
GROUP BY actuo_en.pid, Peliculas.pnombre

 * postgresql+psycopg2://@/postgres


,pnombre,count
0,Avengers:Endgame,3
1,Captain America: Civil War,3
2,Iron Man,2
3,Batman: The Dark Knight,1
4,Batman: The Dark Knight Rises,2
5,Interstellar,1
6,Sherlock Holmes,1
7,Avengers: Age of Ultron,4
8,Doctor Strange,1


Finalmente, si nos interesan solo las películas con tres o más actores, podemos ocupar `HAVING`:

In [28]:
%%sql

SELECT Peliculas.pnombre, COUNT(DISTINCT actuo_en.aid)
FROM actuo_en, Peliculas
WHERE actuo_en.pid = Peliculas.pid
GROUP BY actuo_en.pid, Peliculas.pnombre
HAVING COUNT(DISTINCT actuo_en.aid)>=3

 * postgresql+psycopg2://@/postgres


,pnombre,count
0,Avengers:Endgame,3
1,Captain America: Civil War,3
2,Avengers: Age of Ultron,4


### Pregunta 4

Digamos que ahora nos interesa el número de películas dónde actua cada actor. Aquí queremos devolver el nombre del actor, y el número de peliculas *distintas* dónde actua. Para esto, podemos ocupar:

In [30]:
%%sql

SELECT Actores.anombre, COUNT(DISTINCT actuo_en.pid)
FROM actuo_en, Actores
WHERE actuo_en.aid = Actores.aid
GROUP BY actuo_en.aid, Actores.anombre

 * postgresql+psycopg2://@/postgres


,anombre,count
0,Robert Downey Jr.,5
1,Scarlett Johansson,3
2,Chris Evans,3
3,Christian Bale,2
4,Anne Hathaway,2
5,Paul Bettany,2
6,Benedict Cumberbatch,1


`GROUP BY` por más de un atributo es útil cuando queremos hacer un zoom-in a algunos resultados. Por ejemplo, quizás para cada actor, nos interesa en cuantas películas de cada categoría actuó. Esto podemos lograr con:

In [31]:
%%sql

SELECT Actores.anombre, Peliculas.pcategoria, COUNT(DISTINCT actuo_en.pid)
FROM actuo_en, Actores, Peliculas
WHERE actuo_en.aid = Actores.aid AND actuo_en.pid = Peliculas.pid
GROUP BY actuo_en.aid, Actores.anombre, Peliculas.pcategoria

 * postgresql+psycopg2://@/postgres


,anombre,pcategoria,count
0,Robert Downey Jr.,Mystery,1
1,Robert Downey Jr.,SciFi,4
2,Scarlett Johansson,SciFi,3
3,Chris Evans,SciFi,3
4,Christian Bale,Thriller,2
5,Anne Hathaway,Drama,1
6,Anne Hathaway,Thriller,1
7,Paul Bettany,SciFi,2
8,Benedict Cumberbatch,SciFi,1


### Pregunta 5

Ahora nos interesa devolver, para cada actor, número de personajes distintos que interpreta este actor:

In [38]:
%%sql

SELECT Actores.anombre, COUNT(DISTINCT actuo_en.rol)
FROM actuo_en, Actores
WHERE actuo_en.aid = Actores.aid
GROUP BY actuo_en.aid, Actores.anombre

 * postgresql+psycopg2://@/postgres


,anombre,count
0,Robert Downey Jr.,2
1,Scarlett Johansson,1
2,Chris Evans,1
3,Christian Bale,1
4,Anne Hathaway,2
5,Paul Bettany,2
6,Benedict Cumberbatch,2


Notense que si queremos el nombre de los personajes también, no es necesario ocupar agregación:

In [43]:
%%sql

SELECT Actores.anombre, actuo_en.rol
FROM actuo_en, Actores
WHERE actuo_en.aid = Actores.aid
GROUP BY actuo_en.aid, Actores.anombre, actuo_en.rol
ORDER BY Actores.anombre

 * postgresql+psycopg2://@/postgres


,anombre,rol
0,Anne Hathaway,Amelia Brand
1,Anne Hathaway,Selina Kyle
2,Benedict Cumberbatch,Doctor Strange
3,Benedict Cumberbatch,Dormammu
4,Chris Evans,Steve Rogers
5,Christian Bale,Bruce Wayne
6,Paul Bettany,J.A.R.V.I.S.
7,Paul Bettany,Vision
8,Robert Downey Jr.,Sherlock Holmes
9,Robert Downey Jr.,Tony Stark


### Pregunta 6

Siguiendo con nuestro análisis, ahora nos interesa el número de veces que cada actor ha interpretado a cada personaje. La consulta para conseguir esta información es la siguiente:

In [41]:
%%sql

SELECT Actores.anombre, actuo_en.rol, COUNT(*)
FROM actuo_en, Actores
WHERE actuo_en.aid = Actores.aid
GROUP BY actuo_en.aid, Actores.anombre, actuo_en.rol
ORDER BY Actores.anombre

 * postgresql+psycopg2://@/postgres


,anombre,rol,count
0,Anne Hathaway,Amelia Brand,1
1,Anne Hathaway,Selina Kyle,1
2,Benedict Cumberbatch,Doctor Strange,1
3,Benedict Cumberbatch,Dormammu,1
4,Chris Evans,Steve Rogers,3
5,Christian Bale,Bruce Wayne,2
6,Paul Bettany,J.A.R.V.I.S.,2
7,Paul Bettany,Vision,1
8,Robert Downey Jr.,Sherlock Holmes,1
9,Robert Downey Jr.,Tony Stark,4


### Resumen

En este tutorial aprendimos cómo analizar una base de datos de películas ocupando agregación. Adicionalmente, explicamos cómo ocupar `HAVING` para filtrar grupos definidos por nuestra consulta.